In [11]:
import sys
import pickle

### Load the dictionary containing the dataset
with open("data//final_project_dataset.pkl", "rb") as data_file:
    data_dict = pickle.load(data_file)

UnpicklingError: the STRING opcode argument must be quoted